In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm

from PhaseFunctionedNetwork import PhaseFunctionedNetwork

# set seeds for reproduceability
torch.manual_seed(42)
np.random.seed(42)
rng = np.random.RandomState(42)

In [2]:
# Process Data Mirrored

# Idle Data
X_idle = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Input_Idle_Static.txt'))
Y_idle = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Output_Idle_Static.txt'))
P_idle = np.linspace(0, 2*np.pi, num=72)
print(f"Idle data shape: {X_idle.shape}, {Y_idle.shape}, {P_idle.shape}")

# Jump Flat
X_jump_horizflat = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Input_Jump_HorizFlat.txt'))
Y_jump_horizflat = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Output_Jump_HorizFlat.txt'))
jump_horizflat_1 = np.linspace(0, 2*np.pi, num=50)
jump_horizflat_2 = np.linspace(0, 2*np.pi, num=50)
P_jump_horizflat = np.concatenate((jump_horizflat_1, jump_horizflat_2))
print(f"Idle data shape: {X_jump_horizflat.shape}, {Y_jump_horizflat.shape}, {P_jump_horizflat.shape}")

# Jump Up
X_jump_horizup = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Input_Jump_HorizUp.txt'))
Y_jump_horizup = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Output_Jump_HorizUp.txt'))
jump_horizup_1 = np.linspace(0, 2*np.pi, num=48)
jump_horizup_2 = np.linspace(0, 2*np.pi, num=48)
P_jump_horizup = np.concatenate((jump_horizup_1, jump_horizup_2))
print(f"Idle data shape: {X_jump_horizup.shape}, {Y_jump_horizup.shape}, {P_jump_horizup.shape}")


Idle data shape: (72, 1836), (72, 1767), (72,)
Idle data shape: (100, 1836), (100, 1767), (100,)
Idle data shape: (96, 1836), (96, 1767), (96,)


In [3]:
# Concatenate Data into indivudal X, Y, P Arrays
X = np.concatenate((X_idle, X_jump_horizflat, X_jump_horizup))
print(f"X: {X.shape}")

Y = np.concatenate((Y_idle, Y_jump_horizflat, Y_jump_horizup))
print(f"Y: {Y.shape}")

P = np.concatenate((P_idle, P_jump_horizflat, P_jump_horizup))
print(f"P: {P.shape}")

X: (268, 1836)
Y: (268, 1767)
P: (268,)


In [4]:
# Phase logic
delta_phase = P[1:] - P[:-1]
delta_phase[delta_phase < 0] = (1.0 - P[:-1] + P[1:])[delta_phase < 0]
delta_phase = np.append(delta_phase, 2 * np.pi)

Y = np.concatenate([Y, delta_phase [..., np.newaxis]], axis=-1)

In [105]:
# WEIGH THE JOINT (in particular all the hand and foot joints need to be weighted less)
JOINT_NUM = 144

joint_weights = np.array([
    1,                                                  # HIPS
    1, 1, 1,                                            # LEFT leg
    1e-10, 1e-10, 1e-10, 1,                             # LEFT foot thumb
    1,                                                  # LEFT foot toe base
    1e-10, 1e-10, 1e-10, 1,                             # LEFT foot index
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # LEFT foot middle
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # LEFT foot ring
    1, 1, 1,                                            # RIGHT leg
    1e-10, 1e-10, 1e-10, 1,                             # RIGHT foot thumb
    1,                                                  # RIGHT foot toe base
    1e-10, 1e-10, 1e-10, 1,                             # RIGHT foot index
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # RIGHT foot middle
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # RIGHT foot ring
    1, 1, 1, 1,                                         # SPINE
    1, 1,                                               # LEFT shoulder
    1, 1,                                               # LEFT arm
    1,                                                  # LEFT hand
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # LEFT hand index
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # LEFT hand middle
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # LEFT hand ring
    1, 1e-10, 1,                                        # LEFT wing feathers large
    1, 1e-10, 1,                                        # LEFT wing feathers medium
    1,                                                  # LEFT wing feathers small
    1, 1, 1, 1, 1, 1, 1,                                # NECK
    1,                                                  # HEAD
    1e-10, 1e-10,                                       # JAW
    1e-10, 1e-10, 1e-10, 1e-10, 1e-10, 1e-10, 1e-10,    # TONUGE
    1e-10, 1e-10, 1e-10,                                # LEFT eye
    1e-10, 1e-10, 1e-10,                                # RIGHT eye
    1, 1,                                               # RIGHT shoulder
    1, 1,                                               # RIGHT arm
    1,                                                  # RIGHT hand
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # RIGHT hand index
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # RIGHT hand middle
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # RIGHT hand ring
    1, 1e-10, 1,                                        # RIGHT wing feathers large
    1, 1e-10, 1,                                        # RIGHT wing feathers medium
    1,                                                  # RIGHT wing feathers small
    1, 1, 1, 1, 1, 1, 1, 1, 1,                          # TAIL
    1e-10, 1e-10, 1,                                    # LEFT tail feather
    1e-10, 1e-10, 1,                                    # MIDDLE tail feather         
    1e-10, 1e-10, 1                                     # RIGHT tail feather 
])

# repeat weights for each joint to represent X, Y and Z
joint_weights = joint_weights.repeat(3)

In [118]:
# # Calculate mean and std
# Xmean, Xstd = X.mean(axis=0), X.std(axis=0)
# Ymean, Ystd = Y.mean(axis=0), Y.std(axis=0)

In [107]:
# # PREPROCESS INPUT X
# # lists to keep track of indices for TRAJECTORY
# X_traj_pos_indices = []
# X_traj_dir_indices = []
# X_traj_vel_indices = []
# X_traj_speed_indices = []
# X_traj_style_indices = []

# # number of trajectory elements
# w = 9
# for i in range(0, 107, w):
#     X_traj_pos_indices = np.append(X_traj_pos_indices, range(i,i+2)).astype(int)
#     X_traj_dir_indices = np.append(X_traj_dir_indices, range(i+2,i+4)).astype(int)
#     X_traj_vel_indices = np.append(X_traj_vel_indices, range(i+4,i+6)).astype(int)
#     X_traj_speed_indices = np.append(X_traj_speed_indices, i+6).astype(int)
#     X_traj_style_indices = np.append(X_traj_style_indices, range(i+7,i+9)).astype(int)

# # lists to keep track of indices for JOINTS
# X_joint_pos_indices = []
# X_joint_for_indices = []
# X_joint_up_indices = []
# X_joint_vel_indices = []

# # num of joint elements
# w = 12
# for i in range(108, 1835, w):
#     X_joint_pos_indices = np.append(X_joint_pos_indices, range(i,i+3)).astype(int)
#     X_joint_for_indices = np.append(X_joint_for_indices, range(i+3,i+6)).astype(int)
#     X_joint_up_indices = np.append(X_joint_up_indices, range(i+6,i+9)).astype(int)
#     X_joint_vel_indices = np.append(X_joint_vel_indices, range(i+9,i+12)).astype(int)

# # INPUT Trajectory data
# Xstd[X_traj_pos_indices] = Xstd[X_traj_pos_indices].mean()
# Xstd[X_traj_dir_indices] = Xstd[X_traj_dir_indices].mean()
# Xstd[X_traj_vel_indices] = Xstd[X_traj_vel_indices].mean()
# Xstd[X_traj_speed_indices] = Xstd[X_traj_speed_indices].mean()
# Xstd[X_traj_style_indices] = Xstd[X_traj_style_indices].mean()

# # INPUT Joint data --> This is where we weight the joints
# Xstd[X_joint_pos_indices] = Xstd[X_joint_pos_indices].mean() / (joint_weights * 0.1)
# Xstd[X_joint_for_indices] = Xstd[X_joint_for_indices].mean() / (joint_weights * 0.1)
# Xstd[X_joint_up_indices] = Xstd[X_joint_up_indices].mean() / (joint_weights * 0.1)
# Xstd[X_joint_vel_indices] = Xstd[X_joint_vel_indices].mean() / (joint_weights * 0.1)

In [108]:
# # PREPROCESS OUTPUT Y
# # lists to keep track of indices for TRAJECTORY
# Y_traj_pos_indices = []
# Y_traj_dir_indices = []
# Y_traj_vel_indices = []

# # number of trajectory elements
# w = 6
# for i in range(0, 35, w):
#     Y_traj_pos_indices = np.append(Y_traj_pos_indices, range(i,i+2)).astype(int)
#     Y_traj_dir_indices = np.append(Y_traj_dir_indices, range(i+2,i+4)).astype(int)
#     Y_traj_vel_indices = np.append(Y_traj_vel_indices, range(i+4,i+6)).astype(int)

# # lists to keep track of indices for JOINTS
# Y_joint_pos_indices = []
# Y_joint_for_indices = []
# Y_joint_up_indices = []
# Y_joint_vel_indices = []

# # num of joint elements
# w = 12
# for i in range(36, 1763, w):
#     Y_joint_pos_indices = np.append(Y_joint_pos_indices, range(i,i+3)).astype(int)
#     Y_joint_for_indices = np.append(Y_joint_for_indices, range(i+3,i+6)).astype(int)
#     Y_joint_up_indices = np.append(Y_joint_up_indices, range(i+6,i+9)).astype(int)
#     Y_joint_vel_indices = np.append(Y_joint_vel_indices, range(i+9,i+12)).astype(int)

# # OUTPUT Trajectory data
# Ystd[Y_traj_pos_indices] = Ystd[Y_traj_pos_indices].mean()
# Ystd[Y_traj_dir_indices] = Ystd[Y_traj_dir_indices].mean()
# Ystd[Y_traj_vel_indices] = Ystd[Y_traj_vel_indices].mean()

# # OUTPUT Joint data --> This is where we weight the joints
# Ystd[Y_joint_pos_indices] = Ystd[Y_joint_pos_indices].mean() / (joint_weights * 0.1)
# Ystd[Y_joint_for_indices] = Ystd[Y_joint_for_indices].mean() / (joint_weights * 0.1)
# Ystd[Y_joint_up_indices] = Ystd[Y_joint_up_indices].mean() / (joint_weights * 0.1)
# Ystd[Y_joint_vel_indices] = Ystd[Y_joint_vel_indices].mean() / (joint_weights * 0.1)

# #TODO MAYBE ROOT TRASLATION arr[-3:] --> last three elements

In [5]:
# Calculate mean and std
Xmean, Xstd = X.mean(axis=0), X.std(axis=0)
Ymean, Ystd = Y.mean(axis=0), Y.std(axis=0)

for i in range(Xstd.size):
    if (Xstd[i]==0):
        Xstd[i]=1
for i in range(Ystd.size):
    if (Ystd[i]==0):
        Ystd[i]=1

# save mean / std / min / max
Xmean.astype(np.float32).tofile('C:/Users/Ana/Desktop/dev/pfnn-dev/unity-pfnn/Assets/Dev/Weights/test/Xmean.bin')
Ymean.astype(np.float32).tofile('C:/Users/Ana/Desktop/dev/pfnn-dev/unity-pfnn/Assets/Dev/Weights/test/Ymean.bin')
Xstd.astype(np.float32).tofile('C:/Users/Ana/Desktop/dev/pfnn-dev/unity-pfnn/Assets/Dev/Weights/test/Xstd.bin')
Ystd.astype(np.float32).tofile('C:/Users/Ana/Desktop/dev/pfnn-dev/unity-pfnn/Assets/Dev/Weights/test/Ystd.bin')

# normalize data NOTE ORIGINAL DID THIS AFTER SAVING THE MEANS AND STD AS DONE HERE
X = (X - Xmean) / Xstd
Y = (Y - Ymean) / Ystd

In [6]:
# append phase as additional feature
input = torch.tensor(np.concatenate([X, P [..., np.newaxis]], axis=-1))
target = torch.tensor(Y)

print(f"Input shape {input.shape}")
print(f"Target shape {target.shape}")

dataset = TensorDataset(input, target)

BATCH_SIZE = 32
train_dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

Input shape torch.Size([268, 1837])
Target shape torch.Size([268, 1768])


In [7]:
# ensure GPU is available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")
device = "cuda"

# custom loss function
def loss_func(output, target, model):
    loss = torch.mean((output - target)**2) + model.cost()
    return loss

In [8]:
model = PhaseFunctionedNetwork(input_shape=input.shape[1], output_shape=target.shape[1])
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001)

epochs=3

for epoch in range(epochs):
    model.train()
    loss_list = []
    for i, batch in enumerate(tqdm(train_dataloader)):
        input, target = batch
        input, target = input.to(device), target.to(device)

        # forward pass
        output = model(input)
        loss = loss_func(output, target, model)
        loss_list.append(loss.item())

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {np.average(loss_list)}')


100%|██████████| 9/9 [00:01<00:00,  5.85it/s]


Epoch [1/3], Loss: 0.9808530154905812


100%|██████████| 9/9 [00:01<00:00,  6.69it/s]


Epoch [2/3], Loss: 0.9885938492753331


100%|██████████| 9/9 [00:01<00:00,  6.70it/s]

Epoch [3/3], Loss: 0.9370610110583488


In [9]:
# save weights
model.precompute_and_save_weights()